In [ ]:
!pip transformers
!pip install torchaudio
!pip install jiwer
!pip install datasets
!pip install pyctcdecode kenlm

ERROR: unknown command "transformers"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.2/429.2 kB 43.4 MB/s eta 0:00:00
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184465 sha256=cf25db3db1ce1710fee7401bc53871a9d146061e63efba45db20dc898daace01
  Stored in directory: /root/.cache/pip/wheels/fd/80/e0/18f4148e863fb137bd87e21ee2bf423b81b3ed6989dab95135
Successfully built kenlm


In [ ]:
import multiprocessing as mp

try:
   mp.set_start_method('spawn', force=True)
   print("spawned")
except RuntimeError:
   pass

spawned


In [ ]:
import torch

torch.multiprocessing.set_start_method('spawn', force=True)

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC,Wav2Vec2ProcessorWithLM,Wav2Vec2CTCTokenizer
from transformers import AutoModelForCTC, AutoProcessor

import json
import torchaudio
import torch
import jiwer

# Load the pre-trained model and processor
model_name = "patrickvonplaten/wav2vec2-base-960h-4-gram"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCTC.from_pretrained(model_name).to("cuda")

# Set the LibriSpeech dataset configuration
librispeech_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-clean", download=True)

preprocessor_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/395 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

language_model/attrs.json:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

4-gram.bin:   0%|          | 0.00/3.12G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at patrickvonplaten/wav2vec2-base-960h-4-gram were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at patrickvonplaten/wav2vec2-base-960h-4-gram and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizati

In [ ]:
# Process each example in the dataset
def evaluation(librispeech_dataset, processor):
  predictions = []
  ground_truths = []

  for i in range(len(librispeech_dataset)):
      waveform, utterance, _ = librispeech_dataset[i]

      # Preprocess the audio file
      input_values = processor(waveform.numpy(), sampling_rate = 16_000,
                               return_tensors="pt").input_values.to("cuda")

      # Perform inference
      with torch.no_grad():
          logits = model(input_values).logits

      # Decode the logits to obtain the transcriptions
      #predicted_ids = torch.argmax(logits, dim=-1)
      transcription = processor.batch_decode(logits.cpu().numpy()).text[0]
      print(i)

      predictions.append(transcription)
      ground_truths.append(utterance)

  return predictions, ground_truths

In [ ]:
import torchaudio
from torch.utils.data import Dataset
import os
import shutil

class GenderSubsetDataset(Dataset):
    def __init__(self, root, subset, utterances):
        self.root = root
        self.subset = subset
        self.utterances = utterances
        self.wav_files = [f for f in os.listdir(os.path.join(root, subset)) if f.endswith(".wav")]

    def __len__(self):
        return len(self.wav_files)

    def __getitem__(self, idx):
        wav_file = self.wav_files[idx]
        wav_path = os.path.join(self.root, self.subset, wav_file)
        waveform, sample_rate = torchaudio.load(wav_path)

        # Include the corresponding utterance
        index = int(wav_file.split(".")[0])
        utterance = self.utterances[index]

        return waveform, utterance, wav_file
# Load the LibriSpeech dataset configuration
librispeech_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-clean", download=True)

# Load information from SPEAKERS.TXT
speakers_file_path = "LibriSpeech/SPEAKERS.TXT"  # Replace with the actual path
speaker_info = {}

with open(speakers_file_path, 'r') as file:
    for line in file:
        if line.startswith(';') or not line.strip():
            continue
        fields = line.strip().split('|')
        speaker_id = int(fields[0].strip())
        gender = fields[1].strip()
        speaker_info[speaker_id] = gender


In [ ]:
# Set paths for male and female subsets
male_subset_path = "librispeech_male"
female_subset_path = "librispeech_female"

# Create directories if they don't exist
os.makedirs(male_subset_path, exist_ok=True)
os.makedirs(female_subset_path, exist_ok=True)

utterances_female = {}
utterances_male = {}

# Process the LibriSpeech dataset
for i in range(len(librispeech_dataset)):
    waveform, sample_rate, utterance, speaker_id, _, _ = librispeech_dataset[i]

    # Determine gender based on information from SPEAKERS.TXT
    gender = speaker_info.get(speaker_id, "unknown")

    # Move the audio file to the appropriate subset directory
    if gender == "M":
        destination_path = os.path.join(male_subset_path, f"{i}.wav")
        utterances_male[i] = utterance
    elif gender == "F":
        destination_path = os.path.join(female_subset_path, f"{i}.wav")
        utterances_female[i] = utterance
    else:
        continue  # Skip if gender is unknown

    torchaudio.save(destination_path, waveform, sample_rate)

    print(f"Processed example {i + 1}: Gender - {gender}")

print(f"Finished processing. Male subset: {male_subset_path}, Female subset: {female_subset_path}")

Processed example 1: Gender - M
Processed example 2: Gender - M
Processed example 3: Gender - M
Processed example 4: Gender - M
Processed example 5: Gender - M
Processed example 6: Gender - M
Processed example 7: Gender - M
Processed example 8: Gender - M
Processed example 9: Gender - M
Processed example 10: Gender - M
Processed example 11: Gender - M
Processed example 12: Gender - M
Processed example 13: Gender - M
Processed example 14: Gender - M
Processed example 15: Gender - M
Processed example 16: Gender - M
Processed example 17: Gender - M
Processed example 18: Gender - M
Processed example 19: Gender - M
Processed example 20: Gender - M
Processed example 21: Gender - M
Processed example 22: Gender - M
Processed example 23: Gender - M
Processed example 24: Gender - M
Processed example 25: Gender - M
Processed example 26: Gender - M
Processed example 27: Gender - M
Processed example 28: Gender - M
Processed example 29: Gender - M
Processed example 30: Gender - M
Processed example 3

In [ ]:
male_dataset = GenderSubsetDataset(root=male_subset_path, subset="", utterances=utterances_male)
female_dataset = GenderSubsetDataset(root=female_subset_path, subset="", utterances=utterances_female)

# Evaluate WER for male subset
print("Evaluate WER for Male Subset:")
male_predictions, male_ground_truths = evaluation(male_dataset,processor)
male_average_wer = jiwer.wer(male_predictions, male_ground_truths)
print(f"Average WER for Male Subset: {male_average_wer}\n")


Evaluate WER for Male Subset:


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


0
1
2


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


3
4
5


...

1220
1221
1222


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


1223
1224


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


1225
1226
1227
1228
1229
1230
Average WER for Male Subset: 0.023374299937772246



In [ ]:
# Evaluate WER for female subset
print("Evaluate WER for Female Subset:")
female_predictions, female_ground_truths = evaluation(female_dataset,processor)
female_average_wer = jiwer.wer(female_ground_truths, female_predictions)
print(f"Average WER for Female Subset: {female_average_wer}")

Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


Evaluate WER for Female Subset:
0
1


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


2
3
4
5
6


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


...

1379
1380
1381


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


1382
1383
1384
1385
1386


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


1387
1388
Average WER for Female Subset: 0.028500297265160523


In [ ]:
print(f"Average WER for Male Subset: {male_average_wer}\n")
print(f"Average WER for Female Subset: {female_average_wer}")

Average WER for Male Subset: 0.023374299937772246

Average WER for Female Subset: 0.028500297265160523


In [ ]:
librispeech_other_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-other", download=True)
male_subset_path = "librispeech_male_other"
female_subset_path = "librispeech_female_other"

# Create directories if they don't exist
os.makedirs(male_subset_path, exist_ok=True)
os.makedirs(female_subset_path, exist_ok=True)

utterances_female = {}
utterances_male = {}

# Process the LibriSpeech dataset
for i in range(len(librispeech_other_dataset)):
    waveform, sample_rate, utterance, speaker_id, _, _ = librispeech_other_dataset[i]

    # Determine gender based on information from SPEAKERS.TXT
    gender = speaker_info.get(speaker_id, "unknown")

    # Move the audio file to the appropriate subset directory
    if gender == "M":
        destination_path = os.path.join(male_subset_path, f"{i}.wav")
        utterances_male[i] = utterance
    elif gender == "F":
        destination_path = os.path.join(female_subset_path, f"{i}.wav")
        utterances_female[i] = utterance
    else:
        continue  # Skip if gender is unknown

    torchaudio.save(destination_path, waveform, sample_rate)

    print(f"Processed example {i + 1}: Gender - {gender}")

print(f"Finished processing. Male subset: {male_subset_path}, Female subset: {female_subset_path}")

100%|██████████| 314M/314M [00:25<00:00, 12.9MB/s]


Processed example 1: Gender - M
Processed example 2: Gender - M
Processed example 3: Gender - M
Processed example 4: Gender - M
Processed example 5: Gender - M
Processed example 6: Gender - M
Processed example 7: Gender - M
Processed example 8: Gender - M
Processed example 9: Gender - M
Processed example 10: Gender - M
Processed example 11: Gender - M
Processed example 12: Gender - M
Processed example 13: Gender - M
Processed example 14: Gender - M
Processed example 15: Gender - M
Processed example 16: Gender - M
Processed example 17: Gender - M
Processed example 18: Gender - M
Processed example 19: Gender - M
Processed example 20: Gender - M
Processed example 21: Gender - M
Processed example 22: Gender - M
Processed example 23: Gender - M
Processed example 24: Gender - M
Processed example 25: Gender - M
Processed example 26: Gender - M
Processed example 27: Gender - M
Processed example 28: Gender - M
Processed example 29: Gender - M
Processed example 30: Gender - M
Processed example 3

In [ ]:
male_dataset_other = GenderSubsetDataset(root=male_subset_path, subset="", utterances=utterances_male)
female_dataset_other = GenderSubsetDataset(root=female_subset_path, subset="", utterances=utterances_female)

# Evaluate WER for male subset
print("Evaluate WER for Male Subset:")
male_predictions_other, male_ground_truths_other = evaluation(male_dataset_other,processor)
male_average_wer_other = jiwer.wer(male_predictions_other, male_ground_truths_other)
print(f"Average WER for Male Subset: {male_average_wer_other}\n")

# Evaluate WER for female subset
print("Evaluate WER for Female Subset:")
female_predictions_other, female_ground_truths_other = evaluation(female_dataset_other,processor)
female_average_wer_other = jiwer.wer(female_predictions_other, female_ground_truths_other)
print(f"Average WER for Female Subset: {female_average_wer_other}")

Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


Evaluate WER for Male Subset:
0


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


1
2


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


...

1367
1368
1369
1370


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


1371
1372
1373


Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.


1374
1375
1376
1377
Average WER for Female Subset: 0.05642279480910288


In [ ]:
print(f"Average WER for Male Subset: {male_average_wer_other}\n")
print(f"Average WER for Female Subset: {female_average_wer_other}")


Average WER for Male Subset: 0.07264052941402793

Average WER for Female Subset: 0.05642279480910288
